# Data Perparation

In [199]:
# Import relevant libraries
import dask.dataframe as dd
from datetime import time, timedelta
import pandas as pd
import numpy as np

In [200]:
# Read in dataset
df = dd.read_parquet("../data/0_RAW/parking_ticket_2022.parquet")
df.head(5)

,summons_number,plate_id,registration_state,plate_type,issue_date,violation_code,vehicle_body_type,vehicle_make,issuing_agency,street_code1,...,vehicle_color,unregistered_vehicle,vehicle_year,meter_number,feet_from_curb,violation_post_code,violation_description,no_standing_or_stopping,hydrant_violation,double_parking_violation
0,1457617912,JEB5683,NY,PAS,2021-06-25T00:00:00.000,40,VAN,FORD,P,63430,...,GY,0,2007,-,3,NaN,NaN,NaN,NaN,NaN
1,1457617924,JAN2986,NY,PAS,2021-06-25T00:00:00.000,20,SUBN,DODGE,P,13490,...,BLU,0,2007,-,0,NaN,NaN,NaN,NaN,NaN
2,1457622427,FJH6630,TX,PAS,2021-06-17T00:00:00.000,98,SDN,AUDI,P,79430,...,WHITE,0,0,-,0,NaN,NaN,NaN,NaN,NaN
3,1457638629,RD1Y5N,MO,PAS,2021-06-16T00:00:00.000,98,SDN,TOYOT,P,53130,...,TAN,0,2001,-,0,NaN,NaN,NaN,NaN,NaN
4,1457639580,T503814C,NY,OMT,2021-07-04T00:00:00.000,40,TAXI,HONDA,P,81030,...,WHI,0,2020,-,0,NaN,NaN,NaN,NaN,NaN


In [201]:
# Define column types
ddf = df.astype(dtype={'summons_number':'int32', 'violation_code':'int16'})

In [202]:
# Create violation_datetime
ddf['issue_date'] = dd.to_datetime(ddf['issue_date'])

In [203]:
def get_datetime(row):
    time = row['violation_time']
    date = row['issue_date']
    min = int(time[2:4])
    """
    if time == 'foo':
        hour = np.nan
        min = np.nan"""
    if time[-1] == 'P':
        hour = int(time[:2]) + 12 % 24
    else:
        hour = int(time[:2])
    datetime = date + timedelta(hours=hour, minutes=min)
    return datetime

In [204]:
ddf['datetime'] = ddf.apply(get_datetime, axis=1, meta=str)

/home/yannik/anaconda3/envs/bigdata/lib/python3.9/site-packages/dask/dataframe/core.py:7067: FutureWarning: Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.
  warnings.warn(


In [205]:
ddf.compute()

,summons_number,plate_id,registration_state,plate_type,issue_date,violation_code,vehicle_body_type,vehicle_make,issuing_agency,street_code1,...,unregistered_vehicle,vehicle_year,meter_number,feet_from_curb,violation_post_code,violation_description,no_standing_or_stopping,hydrant_violation,double_parking_violation,datetime
0,1457617912,JEB5683,NY,PAS,2021-06-25,40,VAN,FORD,P,63430,...,0,2007,-,3,NaN,NaN,NaN,NaN,NaN,2021-06-25 01:30:00
1,1457617924,JAN2986,NY,PAS,2021-06-25,20,SUBN,DODGE,P,13490,...,0,2007,-,0,NaN,NaN,NaN,NaN,NaN,2021-06-25 02:25:00
2,1457622427,FJH6630,TX,PAS,2021-06-17,98,SDN,AUDI,P,79430,...,0,0,-,0,NaN,NaN,NaN,NaN,NaN,2021-06-17 20:09:00
3,1457638629,RD1Y5N,MO,PAS,2021-06-16,98,SDN,TOYOT,P,53130,...,0,2001,-,0,NaN,NaN,NaN,NaN,NaN,2021-06-16 18:05:00
4,1457639580,T503814C,NY,OMT,2021-07-04,40,TAXI,HONDA,P,81030,...,0,2020,-,0,NaN,NaN,NaN,NaN,NaN,2021-07-04 22:58:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1457412469,60352ML,NY,COM,2021-06-30,14,VAN,DODGE,P,0,...,0,0,-,0,NaN,NaN,NaN,NaN,NaN,2021-06-30 12:35:00
996,1466155048,NGHJ60,FL,PAS,2021-06-21,17,SDN,VOLKS,X,75530,...,0,0,-,0,NaN,NaN,NaN,NaN,NaN,2021-06-21 07:10:00
997,1466155061,C68NSB,NJ,PAS,2021-06-21,17,SDN,TOYOT,X,75530,...,0,0,-,0,NaN,NaN,NaN,NaN,NaN,2021-06-21 07:15:00
998,1466155670,FKR5645,NC,PAS,2021-06-21,17,SDN,BMW,C,40390,...,0,0,-,0,NaN,NaN,NaN,NaN,NaN,2021-06-21 11:23:00


In [206]:
ddf['datetime'].head()

0   2021-06-25 01:30:00
1   2021-06-25 02:25:00
2   2021-06-17 20:09:00
3   2021-06-16 18:05:00
4   2021-07-04 22:58:00
Name: datetime, dtype: datetime64[ns]

In [207]:
ddf['datetime'] = dd.to_datetime(ddf['datetime'])

In [208]:
ddf.compute()

,summons_number,plate_id,registration_state,plate_type,issue_date,violation_code,vehicle_body_type,vehicle_make,issuing_agency,street_code1,...,unregistered_vehicle,vehicle_year,meter_number,feet_from_curb,violation_post_code,violation_description,no_standing_or_stopping,hydrant_violation,double_parking_violation,datetime
0,1457617912,JEB5683,NY,PAS,2021-06-25,40,VAN,FORD,P,63430,...,0,2007,-,3,NaN,NaN,NaN,NaN,NaN,2021-06-25 01:30:00
1,1457617924,JAN2986,NY,PAS,2021-06-25,20,SUBN,DODGE,P,13490,...,0,2007,-,0,NaN,NaN,NaN,NaN,NaN,2021-06-25 02:25:00
2,1457622427,FJH6630,TX,PAS,2021-06-17,98,SDN,AUDI,P,79430,...,0,0,-,0,NaN,NaN,NaN,NaN,NaN,2021-06-17 20:09:00
3,1457638629,RD1Y5N,MO,PAS,2021-06-16,98,SDN,TOYOT,P,53130,...,0,2001,-,0,NaN,NaN,NaN,NaN,NaN,2021-06-16 18:05:00
4,1457639580,T503814C,NY,OMT,2021-07-04,40,TAXI,HONDA,P,81030,...,0,2020,-,0,NaN,NaN,NaN,NaN,NaN,2021-07-04 22:58:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1457412469,60352ML,NY,COM,2021-06-30,14,VAN,DODGE,P,0,...,0,0,-,0,NaN,NaN,NaN,NaN,NaN,2021-06-30 12:35:00
996,1466155048,NGHJ60,FL,PAS,2021-06-21,17,SDN,VOLKS,X,75530,...,0,0,-,0,NaN,NaN,NaN,NaN,NaN,2021-06-21 07:10:00
997,1466155061,C68NSB,NJ,PAS,2021-06-21,17,SDN,TOYOT,X,75530,...,0,0,-,0,NaN,NaN,NaN,NaN,NaN,2021-06-21 07:15:00
998,1466155670,FKR5645,NC,PAS,2021-06-21,17,SDN,BMW,C,40390,...,0,0,-,0,NaN,NaN,NaN,NaN,NaN,2021-06-21 11:23:00


In [209]:
ddf = ddf.drop(['issue_date', 'violation_time'], axis=1).compute()

In [211]:
# export dataset
ddf.to_parquet("../data/1_PREPARED/parking_ticket_2022.parquet").